<a href="https://colab.research.google.com/github/Sokolov-AA/ML_Study/blob/main/PlatesKaggle_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kaggle

# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  
# # Then move kaggle.json into the folder where the API expects to find it.
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# !kaggle competitions download -c platesv2
# !unzip platesv2.zip
# !unzip plates.zip

# !mv /content/plates/test /content/plates/unknown
# !mkdir /content/plates/test
# !mv /content/plates/unknown /content/plates/test/unknown

# import os
# import shutil

# def split_sample(src_folder, dst_folder_train, dst_folder_validation, validation_count, file_type='.jpg'):
#   os.makedirs(dst_folder_train)
#   os.makedirs(dst_folder_validation)

#   for dirs, folder, files in os.walk(src_folder):
#     images = list(filter(lambda file: file.endswith(file_type), files))
#     for i in range(len(images)):
#       if i < validation_count:
#         shutil.copyfile(src_folder + images[i], dst_folder_validation + images[i])
#       else:
#         shutil.copyfile(src_folder + images[i], dst_folder_train + images[i])


# split_sample('/content/plates/train/cleaned/', 
#              '/content/plates/modified/train/cleaned/', 
#              '/content/plates/modified/validation/cleaned/',
#              validation_count=4,
#              file_type='.jpg'
#              )

# split_sample('/content/plates/train/dirty/', 
#              '/content/plates/modified/train/dirty/', 
#              '/content/plates/modified/validation/dirty/',
#              validation_count=4,
#              file_type='.jpg'
#              )

train_sample_path = '/content/plates/modified/train/'
validation_sample_path = '/content/plates/modified/validation/'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


print("TensorFlow verison:", tf.__version__)

TensorFlow verison: 2.8.2


In [ ]:
seed = 42
batch_size=8

datagen_train = ImageDataGenerator(
        rotation_range=300,
        shear_range=0.2,
        zoom_range=0.4,
        channel_shift_range=255,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

datagen_validation = ImageDataGenerator()

ds_train_flow = datagen_train.flow_from_directory(
    directory=train_sample_path,
    shuffle=True,
    color_mode='rgb',
    class_mode='binary',
    batch_size=batch_size,
    target_size=(224, 224))

ds_validation_flow = datagen_validation.flow_from_directory(
    directory=validation_sample_path,
    shuffle=True,
    color_mode='rgb',
    class_mode='binary',
    batch_size=batch_size,
    target_size=(224, 224))

Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [ ]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)
base_model.trainable = False
#base_model.summary()

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(224,224,3), batch_size=batch_size),
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu', kernel_regularizer='l2'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid', kernel_regularizer='l2')
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=5,
    decay_rate=0.96,
    staircase=True)

model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
  loss="binary_crossentropy",
  metrics=['accuracy'])

In [ ]:
model.fit(
    ds_train_flow,
    validation_data=ds_validation_flow,
    epochs=100
  )

history = model.history

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
ds_test_ = keras.preprocessing.image_dataset_from_directory(
    "/content/plates/test/", 
    image_size=[256, 341],
    shuffle=False
)

ds_test = (
    ds_test_
    .map(resize_image)
    .cache()
)

prediction = model.predict(ds_test)

test_prediction = [prediction[i][0] for i in range(len(prediction))]
submission_df = pd.DataFrame.from_dict({'id': ds_test_.file_paths, 'label': test_prediction})

submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/content/plates/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)

submission_df.head(n=10)

submission_df.to_csv('submission.csv')

In [ ]:
ds_train_all_ = keras.preprocessing.image_dataset_from_directory(
    "/content/plates/train/", 
    image_size=[256, 341],
    shuffle=False
)

ds_train_all = (
    ds_train_all_
    .map(resize_image)
    .cache()
)

prediction_train_all = model.predict(ds_train_all)

train_prediction = [prediction_train_all[i][0] for i in range(len(prediction_train_all))]
submission_df = pd.DataFrame.from_dict({'id': ds_train_all_.file_paths, 'label': train_prediction})

submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/content/plates/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)

submission_df.head(n=40)